# $\mathrm{H} \rightarrow \mathrm{ZZ}$ : Higgs boson discovery
---------------------------------------
by Artur Monsch and Artur Gottmann

Last updated at December 18, 2020

-----------------------------------

In 2012 at CERN the discovery of the previously predicted Higgs boson was made and thus a piece of the Standard Model of particle physics was confirmed. One of the decay channels that led to the discovery was the decay into four leptons. Compared to the other decay channels, this decay channel is ideally suited for analysis, which you now carry out in the form of this notebook.

This notebook is the first part of this analysis and deals mainly with the simulated and measured datasets. The aim is to increase the sensitivity and to achieve a high ratio between the background and the signal. At the end of the task, the significance of the measurement carried out will be estimated. Based on the resulting significance, a first statement about the detection of the Higgs boson can be made. A detailed statistical treatment of the significance up to the combination of measurements to increase the significance will be presented in the second part, which you will probably encounter in the TP2 course.

The inspiration for this exercise is the following [example analysis](http://opendata.cern.ch/record/5500)

In [ ]:
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

sys.path.append("..")

from include.RandomHelper import check_data_state

# Check if all folders in the directory are present and unpacking them
check_data_state(directory="../data") 

# Events
------------------

To get familiar with the signatures of proton-proton collision events recorded by the CMS experiment, have a look at the visualization of the CMS detector together with the examples of recorded
events with the [**Ispy-WebGL web interface**](https://github.com/cms-outreach/ispy-webgl). The various components of the CMS multi-purpose detector, which are used to detect different particles, are shown in the [**detector slice**](https://cds.cern.ch/record/2120661/files/CMSslice_whiteBackground.png?subformat=icon-1440):
<center><img src="https://cds.cern.ch/record/2120661/files/CMSslice_whiteBackground.png?subformat=icon-1440" width=75%></center>


By using Ispy-WebGL, try to understand the functionality of each component by enabling them in the 'Detector' menu, and to select events with interesting signatures from the event collections. Try also to enable and disable 'Physics' objects in the corresponding menu.


<div class="alert alert-info">
Select two examples for each of the decays listed below and save them as an image. How would the typical signature of these individual decays appear in the detector?

- $\mathrm{H} \rightarrow \mathrm{ZZ} \rightarrow 4\ell$
- $\mathrm{H} \rightarrow \gamma \gamma$
- $\mathrm{H} \rightarrow \mathrm{W}^+ \mathrm{W}^- \rightarrow 2\ell 2\nu$

</div>


The switch-on functions of individual detector components and the option to save the individual events as images can be performed within the web interface. To open the event collection, you must first download `TP1_event_collection.ig` located in the `./data/for_event_display/ig_files/` folder locally on your computer. Then, within the event display under the tab "Open" - folder symbol - this file (open local file) can be selected and opened. The images can then be inserted directly into the notebook using `<img src="your img url or path">` in a markdown cell.


> note:
> `.ig` data format is similar to a `.zip` archive and containing one or more event files and is related to ISpy which is used by CMS for displaying events.

With an Internet connection:

In [ ]:
%%html
<iframe src="https://ispy-webgl.web.cern.ch/ispy-webgl/" width="100%" height="700"></iframe>

Without an internet connection: Open the `index.html` from the [Github Repository](https://github.com/cms-outreach/ispy-webgl) locally in a web browser.

The main task in this section is a manual classification of the results according to their decay channels based on the event images. The classification is later automated using software designed for this purpose to analyse a large number of events within a short time instead of looking at each collision event image by image.


The focus of this exercise will be to rediscover the decay of the Higgs boson into two Z bosons, which in turn decay into four charged leptons, $H\rightarrow ZZ\rightarrow 4\ell$.
From all charged leptons, only electrons and muons are used in the analysis, since decays of the Z boson into two $\tau$ leptons, $Z\rightarrow\tau\tau$, are much more difficult to handle,
and it will be more difficult to distinguish the $H\rightarrow ZZ$ signal from backgrounds.


>Detailed Explanation:
>
>The $\tau$ leptons decay before they reach the first tracker layer but it is possible to reconstruct them from the visible final states. The difficulty arises from the additional neutrinos in the $\tau$ lepton decays. Since these neutrinos carry some part of the energy away, the $H\rightarrow ZZ$ peak in ${m}_{4\ell}$ distribution - $\ell$ corresponds in this case to visible decay products of the $\tau$ lepton -
is smeared out and shifted to lower energies. It is therefore much more difficult to distinguish $H\rightarrow ZZ$ from backgrounds (especially $Z\rightarrow 4 \ell$),
if using $\tau$ leptons.

### Your possible solution:
-------------------------------------
* $\mathrm{H} \rightarrow \mathrm{ZZ} \rightarrow 4\ell$    
  (Please insert your notes and include your images here)
* $\mathrm{H} \rightarrow \gamma \gamma$    
  (Please insert your notes and include your images here)
* $\mathrm{H} \rightarrow \mathrm{W}^+ \mathrm{W}^- \rightarrow 2\ell 2\nu$    
  (Please insert your notes and include your images here)
--------------------------------------

# Data format
------------------------

In the course of this exercise, custom classes will be presented to reduce the time and complexity involved in the processing the data sets. These classes are built upon native packages, like `numpy`, `pandas` or `matplotlib`, combining steps required to be done otherwise, if using the native packages, for example to create a histogram.

In general, it is a common approach, which you can also apply to your future analyses: It is not necessary to write everything from scratch based on native packages, each time you would like to process new data sets. Instead, it is useful to create an intermediate layer between your analysis workflows and the existing native packages, usually summarized into an analysis software framework.

The information required for the analysis performed in this exercise is written from the individual events to several files. In this step, a very loose preselection of events was also carried out and only variables necessary for the analysis were written out, including information on leptons. The original data sets are several TB large and a time-efficient processing requires a cluster of worker nodes, on which it must run for several hours. The data sets used in the following are only a few MB in size and can be processed quite fast within this exercise.

The data format used in this exercise, from which all required quantities are taken, is a modified, human-readable `.csv` format.

A collision event consists of information on different quantities, which are separated with `;` for each event record (a row). Each quantity - like **muon_px** - is stored as a string of float or integer numbers separated by `,` to account for the fact, that there can be multiple particles in an event.

In [ ]:
%matplotlib inline
#%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# Background simulation
name_1 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_2el2mu.csv"
name_2 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_4mu.csv"
name_3 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_4el.csv"

# Signal simulation
# name_1 = "../data/for_long_analysis/mc_init/MC_2012_H_to_ZZ_to_4L_2el2mu.csv"
# name_2 = "../data/for_long_analysis/mc_init/MC_2012_H_to_ZZ_to_4L_4mu.csv"
# name_3 = "../data/for_long_analysis/mc_init/MC_2012_H_to_ZZ_to_4L_4el.csv"

dataframe_1 = pd.read_csv(name_1, delimiter=";")
dataframe_2 = pd.read_csv(name_2, delimiter=";")

In [ ]:
dataframe_1

In [ ]:
dataframe_2

About the quantities contained in the data sets:

Quantities with a prefix (**muon_** or **electron_**) are lepton-specific. In data sets containing the mixed channel $2e2\mu$ these prefixes lead to a clear distinction between the two lepton types.

- **run**, **event** and **luminosity_section** are event-specific variables and should ensure the uniqueness of individual events
- **energy** and **charge** are the energies (in GeV) and the electric charges (in 1e units) of the leptons in an event
- (**px**, **py**, **pz**) are the respective momentum components (in GeV) of the detected leptons
- **relPFIso** is the relative isolation of the leptons. It is the sum over the transverse momenta of all non-leptonic particles divided by the transverse momentum of the considered lepton, computed in a speficied cone in $(\Delta\eta, \Delta\phi)$ around the lepton. To avoid inaccurate reconstruction and misidentification with other particles a good relative isolation is important: the smaller the value, the better, leading to less particles surrounding the considered lepton.
- **SIP3d** is the significance of the 3D impact parameter, and **dxy** and **dz** are the impact parameters (in cm) transverse and longitudinal with respect to the beam axis. These quantities allow to distinguish leptons from prompt decays, like from a Z boson, from leptons resulting from decays of long-living particles, like B mesons.

To access quantities of individual particles - muons or electrons in this case - the strings (`str`) with `,` can be converted into lists (`list`) of numbers.
For this purpose, either the `ast` library can be used, or the `split` method of a `string`:

In [ ]:
import ast

px = ast.literal_eval(f"[{dataframe_2.loc[5, 'muon_px']}]")
px = np.array(px, dtype=float)
px

In [ ]:
py = dataframe_2.loc[5, 'muon_py'].split(",")
py = np.array(py, dtype=float)
py

In the two commands above, the fourth collision event record - counting starts with 0 - was accessed with index "5", corresponding to the fourth row in the `.csv` file. In the first command, a list of the x component of the momentum of the muons in this event was created, using `ast` python library. In this case, there are five muons stored. The second command takes a list of the y component of the muon momenta, this time using `split` method of a string. Again, the list has also five elements, corresponding to five muons in the fourth event.

Both lists are sorted in the same way, such that the x and the y components at the same place in the list belong to the same muon.

Based on quantities from the .csv file, new quantities can be constructed. In case of the example above, the transverse momentum `muon_pt` can be constructed from `muon_px` and `muon_py`, using the advantages
of the `numpy` package to perform operations on the entire array:

In [ ]:
pt = np.sqrt(px ** 2 + py ** 2)
pt

An important step in an analysis of collision events is the selection of well reconstructed and identified physics objects, so muons and electrons in this case, with the aim to enrich signal and to suppress background.

One of the first steps in such a selection can be demonstrated on the transverse momenta of the muons, that was constructed with the commands above. It is convenient to require muons with a
transverse momentum greater than 5 GeV to suppress badly reconstructed and misidentified muons.

In [ ]:
pt_minimum_filter = pt > 5
pt_minimum_filter

In [ ]:
pt = pt[pt_minimum_filter]
pt

As before, the strength of `numpy` was utilized: the comparison of an entire array. In the first step, an array of booleans is created, which state, whether the requirement `pt > 5` was fulfilled or not. After that, the array with booleans can be used to reject elements not fulfilling the requirement by using it as an index argument for the `pt` array. As expected, the last entry in the array was removed.


The boolean array acts like a filter on the array with the quantity `pt`. To reject the undesired muons consistently from an event, the filter must be applied to **all** quantities related to muons in the considered event. This filter step must be repeated for each collision event of stored in the data sets, and the processed data sets must be saved again.

To reduce the effort of this time-consuming task significantly, a custom, optimized class was created in the context of this exercise,
that performs the filter steps and creation of new quantities automatically.

# Application of filters with the custom Apply Class
-----------------------------------

During an analysis, it is usually required to create new quantities, select physics objects (muons and electrons in our case) satisfying quality requirements, and filter out collision events, which do not fulfill desired requirements. In that way, only data and information is chosen, which is of interest for the analysis, reducing the size of inputs to analyse.

These tasks were demonstrated in the small example in the previous section:

* We introduced the transverse momentum of the muons `pt` in a collision event by computing it from `px` and `py` of the muons,
* we checked, whether the muons satisfy the `pt > 5` requirement, leading to the selection of 4 out of 5 muons. This would need to be propagated to all quantity records of the collision event.
* Additionally, it can be required, that a collision event should have at least 4 muons with `pt > 5`, otherwise, such an event would need to be discarded.

To cover such tasks, a custom class `Apply` is used, which is presented in this section and comprises an example of a lightweight analysis framework. It can be used to perform different selection, filter and reconstruction steps, to visualize the intermediate results or to save the intermediate data sets.

In the context of this exercise, code templates are given in the following, which you will have to complete in the next sections to move forward through the analysis steps.

First the class needs to be imported:

In [ ]:
from include.processing.Apply import Apply

When creating an `Apply` object, a filter class and calculation class must be passed to `Apply` to make it work properly. This is done with the later used key word arguments `calc_instance` and `filter_instance`. These two passed classes form a snapshot of existing functions at the time of initialization that `Apply` uses in the filter and reconstruction steps. An example is given in the following.

In detail, the two classes must contain the calculation instructions for new variables (`calc_instance`), and the selection criteria of the filters (`filter_instance`). The methods within these classes should be created as stand-alone with `@staticmethod` and collected logically into two classes.

The example from the previous section can be implemented for the `Apply` class as follows:

In [ ]:
class Calc_Start(object):
    
    @staticmethod
    def pt(px, py):
        return np.sqrt(px ** 2 + py ** 2)

class Filter_Start(object):
    
    @staticmethod
    def pt_min(pt, look_for):
        return pt > 7.0 if look_for == "electron" else (pt > 5.0 if look_for == "muon" else None)

The implemented python functions `pt` and `pt_min` inside the classes `Calc_Start` and `Filter_Start` can now be passed to the Apply class as follows:

In [ ]:
process = Apply(file=name_2, nrows=2000,
                calc_instance=Calc_Start, filter_instance=Filter_Start)

For `file` you need to provide the path to the respective record, `nrows` indicates how many rows, i.e. events are read - if no `nrows` is given (`nrows=None`), all rows, i.e. all events are read. For the example it is sufficient to consider a small number of events. For testing purposes of your implementations of filters and new quantities it is usually sufficient to run on a subset of events, so please keep that in mind, when implementing your analysis.

As soon as you converge on your selection and quantities, you should of course run the analysis for all events - more on that later.

The functions implemented in this way (`pt` in `Calc_Start` and `pt_min` in `Filter_Start` in the `Apply` object `process` created above) are alone not sufficient to perform the desired analysis, since as was shown in the previous section, the procedure is applied to one quantity only and not to all that are contained in the event.

Therefore, these functions need to be integrated into already implemented *workflows* of the `Apply` class, which perform the required application of the specified functions on the entire event record.

An overview of all existing workflows can be displayed as follows:

In [ ]:
Apply.help()

A summary of what is being done in a specific workflow can checked as follows:

In [ ]:
Apply.help("filter", "pt_min")

or:

In [ ]:
Apply.help("filter", "pt_exact")

In the example above you see, that the two workflows differ, but make use of similar sets of functions:

* The workflow `"pt_min"` removes the leptons from the event that have transversal momentum below a set threshold. If less than 4 leptons are left (4 electrons, 4 muons, or 2 electrons and 2 muons), then the event is discarded.
* In contrast to that, the workflow `"pt_exact"` discards the whole event if the individual leptons do not meet the exact criteria.

In summary, this example demonstrates the difference between a lepton based filter, which can modify the event content, and an event based filter, which only is used to discard events and does not change the event record in any way.

The call of `Apply.help(workflow, method)` shows the workflows, as well as the required functions. However, since we have already initialized an `Apply` object `process` and passed certain `calc_instance` (`Calc_Start`) and `filter_instance` (`Filter_Start`) to it, it is possible to check if all necessary methods for a workflow are also present in the `process` object:

In [ ]:
process.help("filter", "pt_min")

In [ ]:
process.help("filter", "pt_exact")

In the workflow `"pt_min"` for example only the methods `Calc_Start.pt` and `Filter_Start.pt_min` are used, where `Filter_Start.pt_min` acts as a filter. For this workflow all necessary parts are implemented.

In constrast to that, the workflow `"pt_exact"` still requires the method (the filter) `FilterStudent.pt_exact`, which will be implemented by you in the next sections.


So it is possible to use the `"pt_min"` workflow immediately. There, the transverse momentum `pt` is added implicitly as a method during the workflow.

For demostration purposes, we can also add transverse momentum explicitly beforehand, which does not yet exist in the data set that we called with `name_2`. In an intermediate step already contained in the workflow `"pt_min"`, the transverse momentum `"pt"` is added to the data set `name_2` by executing the required commands given below:

In [ ]:
dataframe_2

In [ ]:
process.add("pt")
process.data

The histogram of such a variable can be displayed by the method `hist(variable, bins, hist_range)`.

In [ ]:
process.hist(variable="pt", bins=50, hist_range=(0, 80))

Although this method is also called `hist`, it differs from the already known `plt.hist` from `matplotlib.pyplot`. Under the hood, `plt.hist` is also used, but in addition, it takes care of proper assignment and conversion of quantities stored as strings in the data set record.

Applying the filter for the minimum transverse momentum and removing all events containing less than four leptons provides the appropriate distribution:

In [ ]:
process.filter("pt_min")

In [ ]:
import matplotlib.pyplot as plt

def pt_hist_helper(title, lepton_number=None):
    process.hist(variable="pt", bins=50, hist_range=(0, 80), lepton_number=lepton_number)
    # you can access the current axis by plt.gca (GetCurrentAxis) or current figure with plt.gcf
    # from here on it is up to you to customize the plot in the way you like
    ax = plt.gca()
    ax.set_title(title)
    ax.set_ylabel("N")
    ax.set_xlabel("$p_T$ in GeV")
    # plt.savefig(f"histogramms/background_4mu_{title.replace('$', '').replace(' ', '_')}.png")
    plt.show()

pt_hist_helper("$p_T$ of all leptons")
pt_hist_helper("$p_T$ of first lepton", 0)
pt_hist_helper("$p_T$ of third and fourth lepton", [2, 3])
pt_hist_helper("$p_T$ of fourth and fifth lepton", [3, 4])

<div class="alert alert-info">
Other variables from the data set(s) can also be visualized in the histogram by the 'hist' method similarly. Please extend the code above to see the impact of the filter step with the requirement on the muon transverse momentum by looking at other variables existing in the event record before and after the filter step. Which variables are changed significantly by requirement?

Do all distributions look like you would expect it? If not, why?
</div>

In [ ]:
# Please enter your code here
# Visualize other quantities in the data record here (before/after the requirement).
# You can also come back here to use code fragments to visualize your further implemented requirements.

# Calculation of important variables
--------------------------------

In this section you should now implement further variables necessary for the analysis, in a similar way as was done for the transverse momentum. The details on implementation of the respective functions are briefly summarised in the docstrings of the function templates given below. Additional information is given on the input arguments and their structure, and how the output should look like to be compatible with the respective workflows of the individual steps of the `Apply` class. Three new quantities have to be calculated in this section of the exercise.

<div class="alert alert-info">

Following the example of the transverse momentum, implement the following subsequently necessary quantities:
  * Pseudorapidity $\eta$
  * Azimuth angle $\phi$
  * The square of invariant mass $m_{\mathrm{inv}}$

Why is it sufficient to calculate the pseudorapidity instead of the rapidity? What is the difference between the two quantities? Also visualise the pseudorapidity and the azimuth angle. Are there special features in the distributions? Explain your observations.

> _Hint_: You can base your visualizations on the example of the transverse momentum. If you create a new `Apply` object, make sure that you always pass the correct `calc_instance` and `filter_instance`. You can also perform this task on a smaller number of events as shown in the example of the previous section (`nrows=2000` in the `Apply` object creation).
</div>

The class to be created now inherits the already known class containing the transverse momentum. In addition, a class will be added later, which contains further already implemented methods. In the end, many of the already implemented methods and the methods you have implemented interact with each other, so it is important that the __method names__, as well as the __names and the number of arguments should not be changed__.

In [ ]:
class CalcStudent(Calc_Start):
    '''
    Class for the calculation of certain quantities that are used for
    the selection requirements or are essential for the reconstruction.
    '''
    
    @staticmethod
    def phi(px, py):
        """
        This function calculates the azimuth angle phi for 
        each lepton in the event. The same array entries of 
        the arguments correspond to the variables of the same 
        leptons.
        
        ------------------------------------------------------
        
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py:  ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
                 
        Exemplary: 
            number of leptons in the event: 5
            
            :param px: np.array([px_1, px_2, px_3, px_4, px_5])
            :param py: np.array([py_1, py_2, py_3, py_4, py_5])
            
            :return: np.array([phi_1, phi_2, phi_3, phi_4, phi_5])
        """
        # Please complete the method with your code, considering 
        # the input and output specifications from the docstring
    
    @staticmethod
    def pseudorapidity(px=None, py=None, pz=None, energy=None):
        """
        Calculates the pseudorapidity for each lepton in an event. 
        The parameter 'energy' is optional and if not necessary 
        can be neglected in the calculation, but should be kept in 
        the argument list as shown above. The same array entries 
        of the arguments correspond to the variables of the same 
        leptons.
        
        ------------------------------------------------------
        
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                    1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        
        Exemplary: 
            number of leptons in the event: 5
            
            :param px: np.array([px_1, px_2, px_3, px_4, px_5])
            :param py: np.array([py_1, py_2, py_3, py_4, py_5])
            :param pz: np.array([pz_1, pz_2, pz_3, pz_4, pz_5])
            :param energy: np.array([e_1, e_2, e_3, e_4, e_5]) (optional)
            
            :return: np.array([psdrapid_1, psdrapid_2, psdrapid_3, psdrapid_4, psdrapid_5])
        """
        # Please complete the method with your code, considering 
        # the input and output specifications from the docstring
        
    @staticmethod
    def invariant_mass_square(px, py, pz, energy=None):
        """
        Calculates the square (!) of the invariant mass. The 
        size of the input arrays depends on how many leptons are 
        present in the event. The parameter 'energy' is optional
        and if not necessary can be neglected in the calculation,
        but should be kept in the argument list as shown above.
        The same array entries of the arguments correspond to the
        variables of the same leptons.
        
        ------------------------------------------------------
        
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                   1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: "float"
        
        Exemplary: 
            number of leptons: 4 energy is not None
            
            :param px: np.array([px_1, px_2, px_3, px_4])
            :param py: np.array([py_1, py_2, py_3, py_4])
            :param pz: np.array([pz_1, pz_2, pz_3, pz_4])
            :param energy: np.array([e_1, e_2, e_3, e_4])
            
            :return: float(mass_square)
            
            or:
            number of leptons: 2 energy is None
            
            :param px: np.array([px_1, px_2])
            :param py: np.array([py_1, py_2])
            :param pz: np.array([pz_1, pz_2])
            :param energy: np.array([e_1, e_2])
            
            :return: float(mass_square)
            
        """
        # Please complete the method with your code, considering 
        # the input and output specifications from the docstring

# Creating selection requirements
------------------------------

As demonstrated with example of a requirement of the minimum necessary transverse momentum, further selection requirements will be implemented and defined by you in this section. You can choose the threshold values for the individual requirements yourself. For this purpose, the visualization of the distributions as you have already seen in previous sections is helpful.

Feel free to test different settings for the selection requirements, since the signal sensitivity of your analysis can vary depending on how a threshold is chosen for a quantity. Therefore, try to figure out a reasonable choice.

A motivation about the threshold values chosen by the CMS Collaboration for the Higgs boson discovery in 2012 can be found in the [**official publication**](https://arxiv.org/pdf/1207.7235.pdf). This paper is a combination of all the decay channels of the Higgs boson, that were studied and combined for the discovery, therefore you can focus one the decay into four leptons while reading it. In case you are interested, feel free to read through the other decay channels - but for the exercise, this is not necessary.

You may also vary the choice of selection thresholds with respect to the values suggested in the paper to try to get a better result. But keep in mind, that the choice of values for the thresholds should be based on studies using the MC simulations and not the measured data to avoid a biased search for a signal peak.

<div class="alert alert-info">

* Complete the methods in the `FilterStudent` class and select the appropriate values for the respective thresholds (see also [**official publication**](https://arxiv.org/pdf/1207.7235.pdf)).  Visualize the application of these selection filters exemplarily - are other variables also influenced by the application of the respective filters?
* A requirement is also applied to the relative isolation. Why is it important to take a closer look at it?
* For the pseudorapidity filter, it is necessary to make a distinction between electrons and muons. Explain this. (_Hint_ : Consider this [**picture**](http://hep.fi.infn.it/CMS/software/ResultsWebPage/Images/Geometry/Tracker_SubDetectors_x_vs_eta.gif))
* Furthermore, a filter is supposed to be made for the reconstructed Z masses. Why is it necessary?

    
> _Hint 1_: You can base your visualisations on the example of the transverse momentum. If you create a new `Apply` object, make sure that you always pass the correct `calc_instance` and `filter_instance`. You can also perform this task on a smaller number of events as shown in the example before (`nrows=2000` in the `Apply` object creation).
>
> _Hint 2_: After completing the `FilterStudent` class, execute the cell with the comment `# Technical necessity` following the implementation of the `FilterStudent` class. After that, you can create an `Apply` object with a complete set of methods for your testing purposes.
</div>

In [ ]:
class FilterStudent(Filter_Start):
    '''
    Class that introduces requirements with certain thresholds chosen by user 
    to restrict the leptons in the events.
    '''
    
    @staticmethod
    def combined_charge(charge, combination_number):
        """
        Tests whether an electrically neutral charge combination is possible.
        With 'combination_number = 4' all four-like combinations of the 
        elements of the electric charge should be formed - also if there are 
        more than four leptons in the event. With 'combination_number = 2' all 
        two-like combinations of the elements of charge - again: there could 
        be more than two leptons that were passed. The size of charge array 
        depends on how many leptons are present in the event.
        
        Note: Either electrons or muons are passed to this method, never
        a mixed collection of leptons. A distinction between "electron"
        and "muon" is therefore not necessary in the implementation.
        
        ------------------------------------------------------
        
        :param charge: ndarray
                       1D array containing data with "int" type.
        :param combination_number: int
                            4 if lepton_type is not "both", 2 else
        :return: bool
        
        Exemplary: 
            combination_number = 4; 
            :param charge: np.array([-1, 1, 1, -1, -1])            
            -> possible four-like combinations are 0 and -2
            -> a neutral electrical combination of four leptons is possible
            :return: True 
            
            or:
            combination_number = 2; 
            :param charge: np.array([-1, -1, -1, -1]) 
            -> possible two-like combination is -2
            -> a neutral electrical combination of two leptons is not possible
            :return: False
        """
        # Please complete the method with your code, considering 
        # the input and output specifications from the docstring
        
    @staticmethod
    def pt_exact(p_t, lepton_type=None):
        """
        This function checks whether the exact requirement for the transverse 
        momentum is met. The values chosen by the CMS Group are:
        (pt>20 GeV: >= 1; pt>10 GeV: >= 2; pt>pt_min: >= 4).
        
        Note: Mixed lepton collections are provided to this method when 
        lepton_type='both' (that appears in the 2el2mu channel). A 
        distinction between "electron" or "muon" is not necessary if the 
        workflow pt_min was executed before. If you want to check again 
        whether the criteria for the minimum transverse momentum is 
        satisfied you need to consider whether lepton_type='electron' or 
        lepton_type='muon'.
        
        ------------------------------------------------------
        
        :param p_t: if lepton_type != "both" 
                        ndarray 
                        1D array containing data with `float` type.
                    if lepton_type == "both"
                        (ndarray, ndarray): (pt_muon, pt_electron)
                        two 1D array containing data with `float` type.
        :param lepton_type: str
                            "muon" (4mu channel) or 
                            "electron" (4el channel) 
                            or "both" (2el2mu channel)
                            
        :return: boolean
                 True or False
        
        Exemplary: 
            :lepton_type: "electron" or "muon" or None
            :param p_t: np.array([22.3, 17.5, 9.4, 6.5])            
            -> the first lepton has a transverse momentum greater than 20 GeV.
            -> the second lepton has a transverse momentum greater than 10 GeV
            a) the workflow pt_min was already carried out so no further 
               steps are necessary 
               -> return True
            b) the workflow pt_min was not carried out before:
               -> if lepton_type == "muon" : 6.5 GeV are greater than the
                  minimum transverse momentum of muons (5 GeV)
               :return: True
               or
               -> if lepton_type == "electron": the last lepton do not pass 
                  minimum transverse momentum threshold of electrons (7 GeV)
               :return: False
            
            lepton_type: "both"
            :param p_t: (np.array([29.4, 16.5]), np.array(10.2, 7.5))
            -> pt_mu, pt_el = p_t
            -> Check the minimal transverse momentum if the workflow pt_min was 
               not carried out before for both transverse momentum arrays 
               (both satisfy the minimum requirement of transverse momentum)
            -> there is one lepton with pt > 20GeV and more than two leptons 
               with pt > 10 GeV
            :return: True
        """
        
        if lepton_type == "both":
            pt_mu, pt_el = p_t
            # Further processing
            # Please complete the method with your code, considering 
            # the input and output specifications from the docstring
        if lepton_type != "both":
            p_t = p_t
            # Further processing
            # Please complete the method with your code, considering 
            # the input and output specifications from the docstring
    
    @staticmethod
    def pseudorapidity(eta, lepton_type):
        """
        This function checks whether the requirement for pseudorapidity is 
        met by the leptons in the event.
        
        
        Note: Either electrons or muons are passed to this method, never
        a mixed collection of leptons. Only a distinction between 
        "electron" and "muon" is necessary. The mixed decay channel is
        split in two and this method is called twice in the corresponding
        workflow.
        
        ------------------------------------------------------
        
        :param eta: ndarray
                    1D array containing data with "float" type.
        :param lepton_type: str
                            "muon" or "electron"
        :return: ndarray
                 1D array containing data with "bool" type.
        
        Exemplary: 
            :param lepton_type: "electron"; 
            :param eta: np.array([1.9, 1.8, -1.7, 1.9, -2.0])            
            -> each lepton satisfies the electron requirement of 
               (abs(eta) < 1.479 | abs(eta) > 1.653) & abs(eta) < 2.5
                   Note: In the range [1,479, 1,653] there are detector 
                         boundaries, which means that no exact reconstruction 
                         for electrons is possible there. Muons do not have 
                         this limitation.
            :return: True
        """
        if lepton_type == "muon":
            # Further processing
            # Please complete the method with your code, considering 
            # the input and output specifications from the docstring
            pass
        if lepton_type == "electron":
            # Further processing
            # Please complete the method with your code, considering 
            # the input and output specifications from the docstring
            pass
    
    @staticmethod
    def impact_parameter(sip3d, dxy, dz):
        """
        This function checks if the variables of the impact parameter 
        meet certain minimum threshold values
        
        Note: No distinction is made between lepton types.
        
        ------------------------------------------------------
        
        :param sip3d: ndarray
                      1D array containing data with "float" type.
        :param dxy: ndarray
                    1D array containing data with "float" type.
        :param dz: ndarray
                   1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "bool" type.
                 
        Exemplary:
            :param sip3d: np.array([sip3d_1, sip3d_2, sip3d_3, sip3d_4, sip3d_5])
            :param dxy: np.array([dxy_1, dxy_2, dxy_3, dxy_4, dxy_5])
            :param dz: np.array([dz_1, dz_2, dz_3, dz_4, dz_5])
            -> Check if spi3d dxy and dz satisfy the corresponding threshold value
            :return: np.array([bool, bool, bool, bool, bool])
        """
        # Please complete the method with your code, considering 
        # the input and output specifications from the docstring
    
    @staticmethod
    def relative_isolation(rel_iso_array):
        """
        Checks if the relative isolation exceeds the value selected as
        threshold for relative isolation or not.
        
        Note: No distinction is made between lepton types.
        
        ------------------------------------------------------
        
        :param rel_iso_array: ndarray
                              1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
                 
        Exemplary:
            :param rel_iso_array: np.array([rel_iso_1, rel_iso_2, rel_iso_3, rel_iso_4])
            -> rel_iso_array < threshold value?
            :return: np.array([bool, bool, bool, bool])
        """
        # Please complete the method with your code, considering 
        # the input and output specifications from the docstring
    
    @staticmethod
    def zz(z1, z2):
        """
        This function checks if the two Z boson masses are within 
        the allowed range. The goal is an off-shell reconstruction. 
        For this reason z1 must be real and z2 virtual. In addition, 
        it should be guaranteed that the masses for z1 and z2 are: 
        z1 > z2.
        
        ------------------------------------------------------

        :param z1: float
                   first Z-Boson mass
        :param z2: float
                   second Z-Boson mass
        :return: bool
                 True or False
        
        Exemplary: 
            :param z1: 91
            :param z2: 45
            -> It satisfy the requirement that z1 > z2
            -> if it satisfy the treshold values for z1 and z2
                :return: True
               else
                :return: False
        """
        # Please complete the method with your code, considering 
        # the input and output specifications from the docstring

The following code combines the inheritance of existing methods and methods implemented by you in such a way that all methods are interoperable inside the workflows and can be used in the Apply class. After executing the cell the `filter_instance=Filter` and the `calc_instance=Calc` can be passed when creating an `Apply` object. If the code in the respective methods is changed, this cell must **always be executed first** to update the methods in `Calc` and `Filter`.
> This kind of "fragmentation" of the `Calc` and `Filter` classes that are passed to `Apply` should be avoided in a framework you create yourself in the future (for example: one `Filter` and `Calc` class with all necessary methods is sufficient). The decision of this "fragmentation" was only made in the context of this task.

In [ ]:
# Technical necessity
from include.processing.CalcAndAllowerInit import FilterInit
from include.processing.CalcAndAllowerInit import CalcInit

FilterInit.a_filter_instance = FilterStudent
FilterInit.a_calc_instance = CalcStudent


class Filter(FilterStudent, FilterInit):
    pass


CalcInit.c_filter_instance = Filter
CalcInit.c_calc_instance = CalcStudent


class Calc(CalcStudent, CalcInit):
    pass

# Application of filters and reconstruction on simulated and measured data sets
---------------------------------------

In this section, the requirements you have previously implemented and new quantities will be applied and calculated for the entire datasets. Also, the actual measurement is now listed here. Therefore, if you are still not satisfied with some thresholds of the requirements, you should change them before you run this section, since this part of the analysis will take some time.

Furthermore, this ensures that you do not see the measurement beforehand and try to match your thresholds of the requirements to the measurement - this would contradict the idea of a blinded analysis, which is optimized before looking at data to avoid bias through subjectivity.

> If you are running this on a machine that has multiple cores available for your use, you can specify that by the argument "multi_cpu=True" and the specification "n_cpu=3" in the Apply object creation (or any other number of CPU cores you want to provide).

For the routine shown below, a help function can be created that lists all records in the folders:

In [ ]:
import os


def collect_all_dataset_paths(directories=None):
    """
    This is a help function to collect all data records in specified directories
    
    :param directories: str or list
                        path of a directory or list of paths to directories
    :return: list
             list of str, containing the dataset paths
    """
    
    if isinstance(directories, str):
        return [os.path.join(directories, file) for file in os.listdir(directories) if ".csv" in file]
    if isinstance(directories, list):
        _tmp = []
        for directory in directories:
            _tmp += collect_all_dataset_paths(directory)
        return _tmp

In [ ]:
files = collect_all_dataset_paths(["../data/for_long_analysis/mc_init/", "../data/for_long_analysis/ru_init/"])
files

The routine that performs all the steps listed in Apply.help() in a possible reasonable order may look like this:
<div class="alert alert-info">

 * Why should it be necessary to perform the filter on the electric charge several times?
 * (optional) Is the order of the applied filters reasonable? If not, alter it.
</div>

In [ ]:
def filter_and_reco_process(file):
    
    # Already known loading process of the individual data records
    process = Apply(file=file, calc_instance=Calc, filter_instance=Filter, multi_cpu=False)
    
    # Name of the record, without the name of the directory in which it is contained
    filename = process.filename
    
    # New directory depending on whether it is MC simulation or measured data
    directory = "../data/for_long_analysis/ru" if "CMS_Run" in filename else "../data/for_long_analysis/mc"
    
    # The application of different requirements
    process.filter("electric_charge")
    
    # If you would like to save the processed records after each filter step, you can do it as follows:
    # process.save(os.path.join(f"{directory}_AfterFirstElectricCharge", filename))
    
    process.filter("relative_isolation")
    process.filter("impact_parameter")
    process.filter("pt_min")
    process.filter("pt_exact")
    process.filter("pseudorapidity")
    
    # the following two filters do a quick check if two or four lepton mass exists in a specific range
    # This filter do not make a complete reconstruction where all necessary variables and those further 
    # restrictions are taken into account. They only serve for a short estimate of the values, whether a 
    # mass could be reconstructed at all in this range or not.
    process.filter("two_lepton_mass")
    process.filter("four_lepton_mass")
    
    # Another use of "electric_charge", can you explain why it is necessary here?
    process.filter("electric_charge")
    
    # After the application of all requirements and the implicit creation of all necessary quantities, 
    # the reconstruction is now carried out. First the Z boson pairs and then the four lepton 
    # invariant mass from the two Z bosons.
    process.save(os.path.join(f"{directory}_beforeZZreco", filename))
    process.reconstruct("zz")
    process.save(os.path.join(f"{directory}_afterZZreco", filename))
    process.reconstruct("four_lepton_mass_from_zz")
    process.save(os.path.join(f"{directory}_afterHreco", filename))
    
    del process

Now you can process all data sets. If you want to use one of your saved samples and then start the processing from there, you can do this by changing the directories in "collect_all_dataset_paths" according to your chosen directories.

In [ ]:
from IPython.display import clear_output

files = collect_all_dataset_paths(["../data/for_long_analysis/mc_init/", "../data/for_long_analysis/ru_init/"])
for file in files:
    filter_and_reco_process(file=file)

# Examination of the final distributions
-----------------------

The idea now is to combine all channels in a histogram and look at some specific variables. The aim is to ensure that there is sufficient agreement between the simulated samples and the measured data. In the previous visualizations, the simulated data was displayed shown in form of histograms without any additional scaling.

In this section, histograms from simulated samples should be scaled to match the integrated luminosity of measured data, and a combination of the three decay channels into a single histogram is performed. This scaling and merging can be expressed as follows for each histogram bin:

$$N_{\mathrm{bin}} = \sum_{i\in\{4\mu, 4e, 2\mu2e\}} N_{\mathrm{bin},i}\frac{\mathcal{L}_{\mathrm{exp}}\sigma_i k}{N_{\mathrm{tot},i}} \quad (*)$$

Where $N_{\mathrm{tot},i}$ is the total number of events from the Monte Carlo simulation of the respective channel, $N_{\mathrm{bin},i}$ is the actual number of events in the considered histogram bin and $\sigma_i$ the cross-section of the respective channel.

The correction factor $k$ ($k=1$ for signal simulation and $k=1.386$ for background simulation) is a scaling factor specific to this simulation, which was only introduced because the background simulation, that was simulated up to the next-to-leading order precision in QCD (NLO). In contrast to that, the analysis you perform requires a precision of next-to-NLO in QCD(NNLO) to account for all effects.

Instead of a new simulation, it turned out, that introducing this global correction factor solves the existing problem of NLO$\rightarrow$NNLO.

The $\mathcal{L}_{\mathrm{exp}}$ is the integrated luminosity of the used measured data and is the same as in the 2012 publication.

To avoid confusion it is important to point out, that this scaling is not done event-wise but is applied to the complete histogram (or more precisely to the histograms of the individual channels).

Furthermore, only the Higgs boson signal simulation for a Higgs boson mass of 125 GeV is used here, opposed to the publication using simulated samples with different mass hypotheses. The reason why this simulation is the appropriate one - taking into account the existing, published measurement - will be explained in the second part of the task.

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)
h = HistOf(mc_dir="../data/for_long_analysis/mc_afterHreco",  # <- Your directory with final datasets (MC Simulation)
           ru_dir="../data/for_long_analysis/ru_afterHreco"   # <- Your directory with final datasets (Measurement)
           )

The four lepton invariant masses can be called similar to `Apply.hist()` - the only difference is that in the class `HistOf` all records in the directories were scaled accordingly before they are combined. 

In [ ]:
h.variable("mass_4l", 37, (70, 181))
ax = plt.gca()
ax.set_xlim(70,181)
ax.set_ylim(0, 18)
ax.set_xlabel(r"$m_{4\ell}$ in GeV")
ax.set_ylabel("Entries")
plt.show()

<div class="alert alert-info">
In this notebook, the same data sets are used as those in the 2012 publication.    
 
* What is the integrated luminosity considered here? 
    
The aim of the analysis is the reconstruction of a Higgs boson, which decays into a pair of an off-shell and an on-shell Z boson in the four lepton channel. 
    
* What is meant by off-shell and how is this motivated? 
* How can one check whether this type of process has been successfully reconstructed on the basis of the existing quantities in the data sets? 

In the histogram of the leptons' invariant masses, the measurement has some uncertainty. This uncertainty appears very asymmetrical for low populated histogram bins. 

* Do you have an explanation for this?

    
(_Hint_: the measurement is a counting process)
</div>

The possible variables that can be examined are:

In [ ]:
from include.processing.ApplyHelper import ProcessHelper
print(ProcessHelper.print_possible_variables("../data/for_long_analysis/mc_afterHreco/MC_2012_H_to_ZZ_to_4L_2el2mu.csv"))

You also have the option to examine the decay channels separately or in combination (`channel=["4e", "4mu", "2e2mu"]`), include only events in the histogram that are within a certain interval of $m_{4\ell}$ (`"mass_4l"`) or the mass of one of the reconstructed Z bosons (`"z1_mass"`, `"z2_mass"`): `filter_by=["mass_4l", (115, 135)]`.

As an example: A look at the distribution of the transverse momentum of the first two leptons in the four muon and four electron decay channels, which was used to reconstruct the four lepton invariant masses in the range of $[115, 135]$, shows that indeed only leptons with a transverse momentum greater than or equal to the values set in the thresholds were used and that one of the measured values shows an excess and does not match the simulation if only the background process would be considered without the signal.

In [ ]:
h.variable("pt", bins=10, hist_range=(0,100), 
           filter_by=["mass_4l", (115, 135)], lepton_number=[0, 1], channel=["4mu", "4el"])
ax = plt.gca()
ax.set_xlabel("$p_T$ in GeV")
ax.set_ylabel("Entries")
plt.show()

In [ ]:
# Here is the place to answer the questions from above.
# You can also modify the examples for the task shown above

Please include your answer here as Markdown Text:

# Estimation of statistical significance
--------------------

You will get to know the idea of the determination of statistical significance in the second part of this exercise.


Nevertheless, a simple estimate for the significance can already be made here by:

$$ Z = \sqrt{-2\left( s+(s+b)\ln\left( \frac{b}{s+b} \right) \right)} \, ,$$

where $b$ is the number of background events and $s$ the number of signal events. The details, how to derive the formula above for the significance $Z$, can be found in [**arXiv:1007.1727**](https://arxiv.org/abs/1007.1727).

<div class="alert alert-info">

* Where in this equation do you take the measured data into account implicitly?
* Estimate the significance of the Higgs boson with the mass of 125 GeV. What statements can you make about this value? 
* Derive a term for significance assuming that $s\ll b$ and interpret your result. Compare it with previous results.
* Does anything change if you use a different number of bins or consider a different mass interval and if so, why?
</div>

In [ ]:
# necessary quantities
_, hist = h.variable("mass_4l", 15, (100, 150))
plt.show()
mc_signal, mc_background, measurement = hist.data["mc_sig"], hist.data["mc_bac"], hist.data["data"]

In [ ]:
# your code for this task part